# Projeto 3

<b>Tema:</b> Estimar o retorno financeiro de um filme

<b>Técnica:</b> Regressão Linear

<b>Grupo:</b> Lucas Leal Vale, Matheus Augusto Soares, Rafael Almada
___

In [40]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
movies=pd.read_csv('movies_metadata.csv', low_memory=False)
imdb=pd.read_csv('ratings_small.csv',low_memory=False)

In [42]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [43]:
df=movies.loc[:, {'id','original_title','original_language','genres','popularity','runtime','budget','revenue'}]
df.head()

,id,budget,genres,revenue,popularity,original_language,runtime,original_title
0,862,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",373554033.0,21.946943,en,81.0,Toy Story
1,8844,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",262797249.0,17.015539,en,104.0,Jumanji
2,15602,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",0.0,11.7129,en,101.0,Grumpier Old Men
3,31357,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",81452156.0,3.859495,en,127.0,Waiting to Exhale
4,11862,0,"[{'id': 35, 'name': 'Comedy'}]",76578911.0,8.387519,en,106.0,Father of the Bride Part II


In [44]:
#Tirando budgets diferente de 0 (budget ainda como string
df=df[df.budget != "0"]
df=df.drop([19730, 29503,35587])

In [45]:
#Transformando em float id e budget
df.id=df.id.astype('float64')
df.budget=df.budget.astype('float64')
df.dtypes

id                   float64
budget               float64
genres                object
revenue              float64
popularity            object
original_language     object
runtime              float64
original_title        object
dtype: object

In [46]:
#Ordenando em ordem crescente de acordo com o id, e filtrando valores 0 das variáveis float
df=df[(df.revenue >0) & (df.budget>0)  & (df.runtime>0)]
df=df.sort_values(['id'], ascending=[True])
df.head()

,id,budget,genres,revenue,popularity,original_language,runtime,original_title
17,5.0,4000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",4300000.0,9.026586,en,98.0,Four Rooms
256,11.0,11000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",775398007.0,42.149697,en,121.0,Star Wars
6232,12.0,94000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",940335536.0,25.497794,en,100.0,Finding Nemo
351,13.0,55000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",677945399.0,48.307194,en,142.0,Forrest Gump
2742,14.0,15000000.0,"[{'id': 18, 'name': 'Drama'}]",356296601.0,20.726578,en,122.0,American Beauty


In [47]:
#Tirando media dos ratings
imdb=imdb.sort_values(['movieId'], ascending=[True])
MediaRating=imdb.drop("userId",1,inplace=True)
MediaRating=imdb.drop("timestamp",1,inplace=True)
imdb.columns

Index(['movieId', 'rating'], dtype='object')

In [48]:
imdb.head()

,movieId,rating
9713,1,4.0
35933,1,1.5
52631,1,5.0
35983,1,2.5
12038,1,4.0


In [ ]:
for 